In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn import preprocessing as pp

In [3]:
from sklearn import cross_validation as cv
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
from sklearn.svm import SVC
from sklearn import tree
from sklearn.cross_validation import cross_val_score

In [6]:
def get_final_output(clf, X, y, X_test, out_file):
    clf.fit(X, y)
    y_ = clf.predict(X_test)
    y_results = pd.DataFrame()
    
    y_results["PassengerId"] = X_test.index
    y_results["Survived"] = y_
    
    if(out_file):
        y_results.to_csv(out_file, encoding='utf-8', index=False)

In [7]:
def report_train_accuracy(clf, X, y):
    clf.fit(X, y)
    y_pred = clf.predict(X)
    return metrics.accuracy_score(y, y_pred)

In [8]:
def report_train_test_accuracy(clf, X, y, test_size, random_state):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state = random_state)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return metrics.accuracy_score(y_test, y_pred)

### Strip input data 

In [360]:
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)
print(train.shape, test.shape)

(891, 11) (418, 10)


In [361]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [362]:
cols_to_drop = ["Name", "Ticket", "Cabin"]

In [363]:
train.drop(cols_to_drop, axis= 1, inplace=True)
test.drop(cols_to_drop, axis= 1, inplace=True)

In [364]:
gender_mapping = {'female':0, 'male':1}
train['Sex'] = train['Sex'].map(gender_mapping)
test['Sex'] = test['Sex'].map(gender_mapping)

In [365]:
embarked_mapping = {'S':0, 'C':1, 'Q': 2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
test['Embarked'] = test['Embarked'].map(embarked_mapping)

In [366]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,14.181209,0.896760,0.981429,55.907576,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,27.000000,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [367]:
train.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000,889.000000
mean,0.383838,2.308642,0.647587,29.699118,0.523008,0.381594,32.204208,0.362205
std,0.486592,0.836071,0.477990,14.526497,1.102743,0.806057,49.693429,0.636157
min,0.000000,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,1.000000,3.000000,1.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,2.000000


#### Dealing with missing values

In [368]:
test.Age.fillna(inplace=True, value=test.Age.mean())
test.Fare.fillna(inplace=True, value=test.Fare.mean())

In [369]:
test.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,2.265550,0.636364,30.272590,0.447368,0.392344,35.627188,0.464115
std,0.841838,0.481622,12.634534,0.896760,0.981429,55.840500,0.685516
min,1.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,23.000000,0.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,30.272590,0.000000,0.000000,14.454200,0.000000
75%,3.000000,1.000000,35.750000,1.000000,0.000000,31.500000,1.000000
max,3.000000,1.000000,76.000000,8.000000,9.000000,512.329200,2.000000


In [370]:
train.dropna(inplace=True)
test.dropna(inplace=True)
print(train.shape, test.shape)

(712, 8) (418, 7)


In [371]:
y = train.Survived
train.drop("Survived", axis= 1, inplace=True)
X = train
X_final = test

In [372]:
print(X.shape, X_final.shape, y.shape)

(712, 7) (418, 7) (712,)


In [373]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

###  using NN to imporve the results 

In [374]:
import keras

In [375]:
keras.__version__

'2.0.9'

In [376]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.optimizers import SGD

In [377]:
batch_size = 16    #we have only 712 elements
num_classes = 2    #Survived or not
epochs=200

In [378]:
test_size = 0.2
random_state = 4

In [379]:
X.max().astype(np.float64)

Pclass        3.0000
Sex           1.0000
Age          80.0000
SibSp         5.0000
Parch         6.0000
Fare        512.3292
Embarked      2.0000
dtype: float64

In [380]:
X_max = X.append(X_final).max().astype(np.float64)
print(X_max)
X_scaled = X/X.max().astype(np.float64)
X_final  = X_final/X.max().astype(np.float64)

Pclass        3.0000
Sex           1.0000
Age          80.0000
SibSp         8.0000
Parch         9.0000
Fare        512.3292
Embarked      2.0000
dtype: float64


In [381]:
X_scaled.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,1.000000,1.0,0.2750,0.2,0.0,0.014151,0.0
2,0.333333,0.0,0.4750,0.2,0.0,0.139136,0.5
3,1.000000,0.0,0.3250,0.0,0.0,0.015469,0.0
4,0.333333,0.0,0.4375,0.2,0.0,0.103644,0.0
5,1.000000,1.0,0.4375,0.0,0.0,0.015713,0.0


In [382]:
y.head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

In [383]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size= test_size, random_state = random_state)

In [384]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569,) (143,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [385]:
y_train = keras.utils.to_categorical(np.array(y_train), num_classes)
y_test = keras.utils.to_categorical(np.array(y_test), num_classes)

#y_train = np.array(y_train)
#y_test = np.array(y_test)

In [386]:
X_train = (np.matrix(X_train))
X_test = (np.matrix(X_test))

In [387]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
print(type(X_train), type(X_test), type(y_train), type(y_test))

(569, 7) (143, 7) (569, 2) (143, 2)
<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [388]:
y_train

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.]])

###  Build the Architecture

In [389]:
#Instatiaate a model
model = Sequential()

In [390]:
#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

In [391]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_69 (Dense)             (None, 70)                560       
_________________________________________________________________
dense_70 (Dense)             (None, 70)                4970      
_________________________________________________________________
dense_71 (Dense)             (None, 70)                4970      
_________________________________________________________________
dense_72 (Dense)             (None, 70)                4970      
_________________________________________________________________
dense_73 (Dense)             (None, 2)                 142       
Total params: 15,612
Trainable params: 15,612
Non-trainable params: 0
_________________________________________________________________


In [392]:
sgd = SGD(lr= 0.01)

In [393]:
#Complie the model for running
model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

In [394]:
model.fit(X_train, y_train, batch_size= batch_size, epochs=180, verbose=1, validation_data=(X_test, y_test))

Train on 569 samples, validate on 143 samples
Epoch 1/180
569/569 [==============================] - 0s 586us/step - loss: 0.6472 - acc: 0.5923 - val_loss: 0.6298 - val_acc: 0.6084
Epoch 2/180
569/569 [==============================] - 0s 146us/step - loss: 0.6289 - acc: 0.5923 - val_loss: 0.6141 - val_acc: 0.6084
Epoch 3/180
569/569 [==============================] - 0s 161us/step - loss: 0.6148 - acc: 0.5923 - val_loss: 0.6001 - val_acc: 0.6084
Epoch 4/180
569/569 [==============================] - 0s 165us/step - loss: 0.6012 - acc: 0.6134 - val_loss: 0.5861 - val_acc: 0.6364
Epoch 5/180
569/569 [==============================] - 0s 163us/step - loss: 0.5885 - acc: 0.6731 - val_loss: 0.5727 - val_acc: 0.6923
Epoch 6/180
569/569 [==============================] - 0s 165us/step - loss: 0.5752 - acc: 0.7135 - val_loss: 0.5600 - val_acc: 0.7972
Epoch 7/180
569/569 [==============================] - 0s 164us/step - loss: 0.5636 - acc: 0.7768 - val_loss: 0.5471 - val_acc: 0.7972
Epoch 8/1

569/569 [==============================] - 0s 169us/step - loss: 0.4445 - acc: 0.8032 - val_loss: 0.4335 - val_acc: 0.7972
Epoch 62/180
569/569 [==============================] - 0s 161us/step - loss: 0.4465 - acc: 0.7979 - val_loss: 0.4208 - val_acc: 0.8042
Epoch 63/180
569/569 [==============================] - 0s 163us/step - loss: 0.4423 - acc: 0.8067 - val_loss: 0.4172 - val_acc: 0.8252
Epoch 64/180
569/569 [==============================] - 0s 165us/step - loss: 0.4438 - acc: 0.8049 - val_loss: 0.4204 - val_acc: 0.8042
Epoch 65/180
569/569 [==============================] - 0s 165us/step - loss: 0.4467 - acc: 0.7979 - val_loss: 0.4213 - val_acc: 0.8112
Epoch 66/180
569/569 [==============================] - 0s 162us/step - loss: 0.4438 - acc: 0.7979 - val_loss: 0.4172 - val_acc: 0.8112
Epoch 67/180
569/569 [==============================] - 0s 94us/step - loss: 0.4405 - acc: 0.8032 - val_loss: 0.4237 - val_acc: 0.8112
Epoch 68/180
569/569 [==============================] - 0s 162

569/569 [==============================] - 0s 165us/step - loss: 0.4273 - acc: 0.8190 - val_loss: 0.4116 - val_acc: 0.8252
Epoch 122/180
569/569 [==============================] - 0s 166us/step - loss: 0.4277 - acc: 0.8137 - val_loss: 0.4005 - val_acc: 0.8462
Epoch 123/180
569/569 [==============================] - 0s 162us/step - loss: 0.4273 - acc: 0.8207 - val_loss: 0.4034 - val_acc: 0.8392
Epoch 124/180
569/569 [==============================] - 0s 91us/step - loss: 0.4247 - acc: 0.8120 - val_loss: 0.4001 - val_acc: 0.8322
Epoch 125/180
569/569 [==============================] - 0s 164us/step - loss: 0.4289 - acc: 0.8190 - val_loss: 0.4041 - val_acc: 0.8322
Epoch 126/180
569/569 [==============================] - 0s 163us/step - loss: 0.4266 - acc: 0.8207 - val_loss: 0.4504 - val_acc: 0.8042
Epoch 127/180
569/569 [==============================] - 0s 164us/step - loss: 0.4304 - acc: 0.8225 - val_loss: 0.4067 - val_acc: 0.8252
Epoch 128/180
569/569 [==============================] -

In [395]:
score = model.evaluate(X_test, y_test, verbose=0)

In [396]:
print('Test Loss', score[0])
print('Test Accuracy', score[1])

Test Loss 0.395596502127
Test Accuracy 0.853146853564


### Retrain the model 

In [397]:
X_scaled = np.matrix(X_scaled)
y_scaled = keras.utils.to_categorical(np.array(y), num_classes)
print(type(X_scaled), type(y_scaled))

<class 'numpy.matrixlib.defmatrix.matrix'> <class 'numpy.ndarray'>


In [398]:
model = Sequential()

#Add later to it
model.add(Dense(70, activation="relu", input_shape=(7,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(70, activation="relu", input_shape=(70,)))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
             optimizer = sgd,
             metrics =['accuracy'])

model.fit(X_scaled, y_scaled, batch_size= batch_size, epochs=220, verbose=1)

Epoch 1/220
712/712 [==============================] - 0s 492us/step - loss: 0.6825 - acc: 0.5393
Epoch 2/220
712/712 [==============================] - 0s 140us/step - loss: 0.6573 - acc: 0.5955
Epoch 3/220
712/712 [==============================] - 0s 141us/step - loss: 0.6446 - acc: 0.5955
Epoch 4/220
712/712 [==============================] - 0s 142us/step - loss: 0.6323 - acc: 0.5955
Epoch 5/220
712/712 [==============================] - 0s 142us/step - loss: 0.6193 - acc: 0.5955
Epoch 6/220
712/712 [==============================] - 0s 141us/step - loss: 0.6045 - acc: 0.5955 0s - loss: 0.6119 - acc: 0.614
Epoch 7/220
712/712 [==============================] - 0s 147us/step - loss: 0.5889 - acc: 0.6517
Epoch 8/220
712/712 [==============================] - 0s 142us/step - loss: 0.5737 - acc: 0.7472
Epoch 9/220
712/712 [==============================] - 0s 133us/step - loss: 0.5601 - acc: 0.7823
Epoch 10/220
712/712 [==============================] - 0s 141us/step - loss: 0.5474 - 

712/712 [==============================] - 0s 144us/step - loss: 0.4228 - acc: 0.8272
Epoch 83/220
712/712 [==============================] - 0s 141us/step - loss: 0.4222 - acc: 0.8258
Epoch 84/220
712/712 [==============================] - 0s 140us/step - loss: 0.4230 - acc: 0.8216
Epoch 85/220
712/712 [==============================] - 0s 142us/step - loss: 0.4176 - acc: 0.8301
Epoch 86/220
712/712 [==============================] - 0s 143us/step - loss: 0.4212 - acc: 0.8272
Epoch 87/220
712/712 [==============================] - 0s 140us/step - loss: 0.4208 - acc: 0.8272
Epoch 88/220
712/712 [==============================] - 0s 145us/step - loss: 0.4188 - acc: 0.8287
Epoch 89/220
712/712 [==============================] - 0s 137us/step - loss: 0.4198 - acc: 0.8272
Epoch 90/220
712/712 [==============================] - 0s 144us/step - loss: 0.4206 - acc: 0.8174
Epoch 91/220
712/712 [==============================] - 0s 144us/step - loss: 0.4222 - acc: 0.8272
Epoch 92/220
712/712 [=

712/712 [==============================] - 0s 144us/step - loss: 0.4040 - acc: 0.8385
Epoch 163/220
712/712 [==============================] - 0s 139us/step - loss: 0.3988 - acc: 0.8427
Epoch 164/220
712/712 [==============================] - 0s 141us/step - loss: 0.3982 - acc: 0.8371 0s - loss: 0.4278 - acc: 0.827
Epoch 165/220
712/712 [==============================] - 0s 143us/step - loss: 0.4018 - acc: 0.8329
Epoch 166/220
712/712 [==============================] - 0s 145us/step - loss: 0.3987 - acc: 0.8230
Epoch 167/220
712/712 [==============================] - 0s 143us/step - loss: 0.4021 - acc: 0.8301
Epoch 168/220
712/712 [==============================] - 0s 142us/step - loss: 0.4063 - acc: 0.8244
Epoch 169/220
712/712 [==============================] - 0s 142us/step - loss: 0.4026 - acc: 0.8357 0s - loss: 0.3897 - acc: 0.838
Epoch 170/220
712/712 [==============================] - 0s 141us/step - loss: 0.4042 - acc: 0.8357
Epoch 171/220
712/712 [=============================

In [399]:
X_final = np.matrix(X_final)

print(X_final.shape, type(X_final), )

(418, 7) <class 'numpy.matrixlib.defmatrix.matrix'>


In [400]:
y_final_pred = model.predict_classes(X_final)

418/418 [==============================] - 0s 282us/step


In [401]:
y_final_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [420]:
y_final = pd.DataFrame()

In [421]:
print(y_final_pred.shape, y_final.shape)

(418,) (0, 0)


In [422]:
y_final.head()

""


In [423]:
y_final["PassengerId"] = test.index
y_final["Survived"] = pd.Series(y_final_pred)

In [424]:
y_final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [425]:
y_final.to_csv("titanic_results_06_nn.csv", encoding='utf-8', index=False)